# Clean News 

# cleaning policies

- pl.col('authors').str.contains('Advertis Web')  
- 

In [2]:
import polars as pl
import pandas as pd

In [8]:
df = pd.read_csv('/home/sebacastillo/willow/output/news_narcotráfico_related_2023-08-09_1816.csv')

In [9]:
df.head()

,date_extract,date_article,content,link,authors
0,2023-08-09 18:15:38,2023-08-09T00:00:00.000000,"9 Ago, 2023 Por María Laura Balonga Fuentes j...",https://www.infobae.com/sociedad/policiales/20...,María Laura Balonga
1,2023-08-09 18:15:38,2023-08-09T00:00:00.000000,"9 Ago, 2023 Los dos hermanos que fueron deteni...",https://www.infobae.com/sociedad/policiales/20...,n-a
2,2023-08-09 18:15:38,2023-08-09T00:00:00.000000,"9 Ago, 2023 La mamá de Morena Domínguez —la ne...",https://www.infobae.com/sociedad/policiales/20...,n-a
3,2023-08-09 18:15:38,2023-08-09T00:00:00.000000,"9 Ago, 2023 Por Walter Vazquez Vecinos de Lan...",https://www.infobae.com/sociedad/policiales/20...,Walter Vazquez
4,2023-08-09 18:15:38,2023-08-09T00:00:00.000000,"9 Ago, 2023 Por Eduardo Menegazzi En un hote...",https://www.infobae.com/politica/2023/08/09/sc...,Eduardo Menegazzi


In [12]:
df['date_extract'] = pd.to_datetime(df['date_extract'])
df['date_article'] = pd.to_datetime(df['date_article'])
print(df.head(1))

         date_extract date_article  \
0 2023-08-09 18:15:38   2023-08-09   

                                             content  \
0  9 Ago, 2023 Por  María Laura Balonga Fuentes j...   

                                                link              authors  
0  https://www.infobae.com/sociedad/policiales/20...  María Laura Balonga  


In [14]:
print(df['content'][0])

9 Ago, 2023 Por  María Laura Balonga Fuentes judiciales confirmaron a Infobae que son mayores los dos sospechosos del crimen de Morena Domínguez, la nena de 11 años que murió tras haber sido atacada por motochorros a metros de la escuela de Lanús a la que iba a esta mañana. Ambos están detenidos y son hermanos. “Tienen 25 y 28 años”, indicaron las fuentes consultadas por este medio sobre los responsables del asesinato de la nena que iba a la escuela Almafuerte N°60, ubicada en la calle Molinedo al 3.200, en Villa Giardino, en la zona Oeste del partido de Lanús. Te puede interesar: Tras el crimen de Morena, habló la mamá desde Salta: “Es un momento muy duro, pido justicia” De apellido Madariaga y oriundos de Villa Caraza, los dos sospechosos tienen antecedentes penales. El menor tiene una causa por tentativa de robo de 2019; mientras que el mayor, al menos, tres: una por encubrimiento agravado (2019) por la que accedió a la libertad un año después; otra por encubrimiento (2021); y la úl

In [21]:
df2 = view_string(df['content'][0])

# Polars

In [17]:
import polars as pl
df = pl.read_csv('/home/sebacastillo/willow/data/portals.csv')

In [10]:
def view_string(long_string, chunk_size=100):     
    return [long_string[i:i+chunk_size] for i in range(0, len(long_string), chunk_size)]

In [11]:
topic = 'narcotráfico'

In [14]:
df.filter(pl.col('topic') == topic).select('newsportalurl').to_series().to_list()
df.head()

state,city,newsportalname,newsportalurl,topic
str,str,str,str,str
"""Buenos Aires""","""La Plata""","""El Día""","""https://www.el…","""narcotráfico"""
"""Buenos Aires""","""Mar del Plata""","""La Capital""","""https://www.la…","""narcotráfico"""
"""CABA""","""Buenos Aires""","""Clarín""","""https://www.cl…","""narcotráfico"""
"""Chaco""","""Resistencia""","""Diario Norte""","""https://www.di…","""narcotráfico"""
"""Chaco""","""Sáenz Peña""","""Diario Chaco""","""https://www.di…","""narcotráfico"""


In [15]:
df = df.filter(
    pl.col('topic') == topic
    )
df.head(1)


state,city,newsportalname,newsportalurl,topic
str,str,str,str,str
"""Buenos Aires""","""La Plata""","""El Día""","""https://www.el…","""narcotráfico"""


In [147]:
df['state'][0]

'Buenos Aires'

# From Raw News to Docs

## Select column

In [32]:
q = (
    pl.scan_csv(filepath)
    .select(
        ['content','authors']
    )
)
q.collect()

content,authors
str,str
"""Estimado lecto…","""Diario El Dia …"
"""Estimado lecto…","""Diario El Dia …"
"""Estimado lecto…","""Diario El Dia …"
"""Estimado lecto…","""Diario El Dia …"
"""Estimado lecto…","""Diario El Dia …"
"""Miércoles,09 d…","""n-a"""
"""Miércoles,09 d…","""n-a"""
"""Dólar blue Dón…","""Gustavo Berón-…"
"""Dólar blue Dón…","""n-a"""


## Filter Row with Regex

In [92]:
# Positive
false_advertis = (
    pl.scan_csv(filepath)
    .filter(
        pl.col('authors').str.contains('Advertis Web')       
    )
    .select(
        [
            'link'
        ]
    )
).collect().to_pandas()

In [93]:
false_advertis

,link
0,https://www.elonce.com/secciones/policiales/76...
1,http://superdeportivo.elonce.com/secciones/pol...
2,https://www.diarioriouruguay.com.ar/policiales...


In [75]:
# Positive
false_clickaqui= (
    pl.scan_csv(filepath)
    .filter(
        pl.col('content').str.contains('Click aquí')       
    )
    .select(
        [
            'link',
            'content'
        ]
    )
).collect().to_pandas()
false_clickaqui['link'][0]

'https://www.elonce.com/secciones/policiales/766331-pact-una-cita-por-tinder-lo-drogaron-desvalijaron-y-est-grave.htm'

In [42]:
# Negative
q = (
    pl.scan_csv(filepath)
    .filter(
        pl.col('authors').str.contains('n-a').is_not()        
    )
    .select(
        [
            'authors'
        ]
    )
)
q.collect().to_series().value_counts()

authors,counts
str,u32
"""El Tucumano""",2
"""Ezequiel Miño""",1
"""Gustavo Berón-…",2
"""Daiana Larrat""",3
"""Guido Berrini""",3
"""Advertis Web T…",3
"""Redacción El""",2
"""Diario El Dia …",10
"""Florencia Ojed…",4


## Length of string

In [62]:
q = (
    pl.scan_csv(filepath)
    .select(
    [
        'content',
        pl.col('content').str.lengths().alias('content_length'),        
    ]
    )
).sort('content_length')
q.collect()

content,content_length
str,u32
"""Click aquí par…",342
"""Click aquí par…",342
"""Click aquí par…",342
"""📌La enfermera …",349
"""📌La enfermera …",349
"""La PFA realizó…",561
"""La PFA realizó…",561
"""Click aquí par…",587
"""Se trata del “…",865


## Split column

In [84]:
authors = (
    df
    .select(
    [
        'authors',
        pl.col('authors').str.split(' ').alias('splitname'),
    ])
)

In [86]:
authors

authors,splitname
str,list[str]
"""Diario El Dia …","[""Diario"", ""El"", … ""Dia""]"
"""Diario El Dia …","[""Diario"", ""El"", … ""Dia""]"
"""Diario El Dia …","[""Diario"", ""El"", … ""Dia""]"
"""Diario El Dia …","[""Diario"", ""El"", … ""Dia""]"
"""Diario El Dia …","[""Diario"", ""El"", … ""Dia""]"
"""n-a""","[""n-a""]"
"""n-a""","[""n-a""]"
"""Gustavo Berón-…","[""Gustavo"", ""Berón-Candela"", … ""Lynch""]"
"""n-a""","[""n-a""]"


In [77]:
# Negative
(
    pl.scan_csv(filepath)
    .filter(
        pl.col('authors').str.contains('n-a').is_not()        
    )
    .select(
        [
            'authors'
        ]
    )
).collect().to_pandas()

,authors
0,Diario El Dia De La Plata Www.Eldia.Com-El Dia
1,Diario El Dia De La Plata Www.Eldia.Com-El Dia
2,Diario El Dia De La Plata Www.Eldia.Com-El Dia
3,Diario El Dia De La Plata Www.Eldia.Com-El Dia
4,Diario El Dia De La Plata Www.Eldia.Com-El Dia
...,...
90,El Día
91,El Día
92,Jorgelina Hernández
93,Daniel Quilodrán


### Transformation

In [105]:
df.columns

['date_extract', 'date_article', 'content', 'link', 'authors']

In [18]:
df = pl.read_csv('/home/sebacastillo/willow/output/news_narcotráfico_related_2023-08-09_1951.csv')
df = (
    df.with_columns([
        pl.col('date_extract').str.strptime(
            pl.Date,
            format='%Y-%m-%d %H:%M:%s', strict=True
        ),        

        pl.col('link').str.replace(r"https://www.", "").alias('new'),
        #pl.col('reference').str.replace_all(r"id-", ""),
        # ----------------
        #pl.col('reference').str.split(by=";")
        ##.struct.rename_fields(["first_part", "second_part"])
        ##.alias("fields"),
    ])
)
df[0]

date_extract,date_article,content,link,authors,new
date,str,str,str,str,str
2023-08-09,"""2023-08-09T00:…","""Estimado lecto…","""https://www.el…","""Diario El Dia …","""eldia.com/nota…"


In [19]:
isinstance(df, pl.DataFrame)

True

In [30]:
text = df['content'][0]

In [31]:
view_string(text)

['Estimado lector, muchas gracias por su interés en nuestras notas. Hemos incorporado el registro con ',
 'el objetivo de mejorar la información que le brindamos de acuerdo a sus intereses. Para más informac',
 'ión haga clic aquí Suplementos > Avisos > Servicios > Entretenimientos > Otros sitios > \n15 °C La Pl',
 'ata\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\tMiércoles 9 de Agosto, 2023\n\t\t\t\t\t\t\t\t\t\t \n La muerte de una niña en La',
 'nús tras ser víctima de un ataque de motochorros atravesó la agenda electoral y llevó a los precandi',
 'datos bonaerenses de las principales fuerzas que competirán en las PASO del domingo próximo, Unión p',
 'or la Patria (UxP) y Juntos por el Cambio (JxC), a suspender sus actos de cierre de campaña previsto',
 's para hoy y mañana. Morena Domínguez, de 11 años, murió hoy luego de ser asaltada por motochorros, ',
 'que la atacaron para robarle sus pertenencias cuando caminaba hacia su colegio en la localidad de Vi',
 'lla Diamant

# Text Sumarization

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_str = "IIC/mt5-spanish-mlsum"
tokenizer = AutoTokenizer.from_pretrained(model_str)
model = AutoModelForSeq2SeqLM.from_pretrained(model_str)

In [33]:
input_ids = tokenizer(text, return_tensors="pt").input_ids
output_ids = model.generate(input_ids, max_new_tokens=200, penalty_alpha=0.6, top_k=4,)[0]
print(tokenizer.decode(output_ids, skip_special_tokens=True))

La muerte de una niña en Lanús moviliza a precandidatos a las PASO. El ataque de motochorros moviliza a vecinos y familiares de la víctima


In [34]:
text = df['content'][1]
view_string(text)

['Estimado lector, muchas gracias por su interés en nuestras notas. Hemos incorporado el registro con ',
 'el objetivo de mejorar la información que le brindamos de acuerdo a sus intereses. Para más informac',
 'ión haga clic aquí Suplementos > Avisos > Servicios > Entretenimientos > Otros sitios > \n15 °C La Pl',
 'ata\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\tMiércoles 9 de Agosto, 2023\n\t\t\t\t\t\t\t\t\t\t         \n          El colombi',
 'ano fue condenado a 45 años de prisión en EE UU. Con un frondoso prontuario que incluye múltiples as',
 'esinatos, se lo ha comparado con Pablo Escobar\n       El narcotraficante Otoniel, detenido en bogotá',
 ' / AP Juan Sebastian SERRANO    BOGOTÁ AFP Dairo Antonio Úsuga, “Otoniel”, cuya condena a 45 años de',
 ' cárcel por narcotráfico fue anunciada ayer en un tribunal de EE UU, escaló las filas de varios grup',
 'os armados en Colombia hasta convertirse en el mayor barón de la cocaína de este siglo. Probablement',
 'e pase el 

In [28]:
input_ids = tokenizer(text, return_tensors="pt").input_ids
output_ids = model.generate(input_ids, max_new_tokens=100)[0]
print(tokenizer.decode(output_ids, skip_special_tokens=True,   ))

El colombiano Otoniel, el mayor barón de la cocaína de este siglo. El colombiano, condenado a 45 años de cárcel por narcotráfico, se ha comparado con Pablo Escobar


In [35]:
input_ids = tokenizer(text, return_tensors="pt").input_ids
output_ids = model.generate(input_ids, max_new_tokens=200, penalty_alpha=0.6, top_k=4,)[0]
print(tokenizer.decode(output_ids, skip_special_tokens=True))

El colombiano Otoniel, el mayor barón de la cocaína de este siglo. El colombiano, condenado a 45 años de cárcel por narcotráfico, vuelve a las filas de varios grupos armados
